In [1]:
import pandas as pd
import numpy as np
#import turicreate as tc

import scipy
import math
import random

import sklearn
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds


In [2]:
posts = pd.read_csv('posts.csv')
user_rating = pd.read_csv('userRatings.csv', usecols = ['user_id', 'post_id', 'Ratings'])
following = pd.read_csv('following.csv',index_col = 'my_id')

In [3]:
following.head()

,follower_id,staus
my_id,,
3,6,1
6,3,1
3,2,1
3,7,1
7,2,1


In [4]:
posts.head()

,user_id,title,content,tags,slug,created_at,updated_at,image,status_id,action,post_id
0,2077,What i have learnt so far on HTML,I learnt how to use the table tag as i have us...,NaN,what-i-have-learnt-so-far-on-html-985,2019-08-25 19:39:01,2019-08-28 11:30:00,1,NaN,NaN,NaN
1,1719,HTML BEGINS HERE,"I am on this journey with start.ng, and here ...",Technology,html-begins-here-4d6,2019-08-25 19:44:36,2019-08-28 11:30:00,1,NaN,NaN,NaN
2,1310,My Laziness In The Open,I have not been attending classes on the HNG c...,NaN,my-laziness-in-the-open-029,2019-08-25 19:50:33,2019-08-28 11:30:00,1,NaN,NaN,NaN
3,1787,MY TASK 2,My journey on **StartNG** pre-internship progr...,NaN,my-task-2-649,2019-08-25 19:51:36,2019-08-28 11:30:00,1,NaN,NaN,NaN
4,167,Task 2,"A Summary on The “idongesit.html” CV, Its Str...",NaN,task-2-7e5,2019-08-25 19:51:39,2019-08-28 11:30:00,1,NaN,NaN,NaN


In [5]:
#Let drop some columns
drop_columns = ['tags', 'image','status_id','action', 'updated_at','post_id', 'created_at','slug']

df_posts = posts.drop(posts[drop_columns],axis = 1)

In [6]:
df_posts.head()

,user_id,title,content
0,2077,What i have learnt so far on HTML,I learnt how to use the table tag as i have us...
1,1719,HTML BEGINS HERE,"I am on this journey with start.ng, and here ..."
2,1310,My Laziness In The Open,I have not been attending classes on the HNG c...
3,1787,MY TASK 2,My journey on **StartNG** pre-internship progr...
4,167,Task 2,"A Summary on The “idongesit.html” CV, Its Str..."


In [7]:
user_rating.head()

,user_id,post_id,Ratings
0,1,995.0,5
1,1,355.0,2
2,1,996.0,4
3,1,996.0,5
4,1,996.0,4


In [8]:
user_df = pd.merge(df_posts, user_rating, on='user_id')

user_df .head()

,user_id,title,content,post_id,Ratings
0,2191,What you have learned so far,"1. I have learnt how to publish and article, ...",143.0,3
1,2191,What you wish to still learn,1. I want more insight on clodinary and so on,143.0,3
2,2191,What you wish should be improved,1. For now am OK with the setting,143.0,3
3,4,Good Afternoon People,Hey,999.0,4
4,4,Good Afternoon People,Hey,278.0,3


In [9]:
user_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 622 entries, 0 to 621
Data columns (total 5 columns):
user_id    622 non-null int64
title      622 non-null object
content    622 non-null object
post_id    622 non-null float64
Ratings    622 non-null int64
dtypes: float64(1), int64(2), object(2)
memory usage: 29.2+ KB


In [10]:
user_df['title'].value_counts()

BUILD AROUND YOUR DISCOVERY MECHANISM                           35
THE SUNFLOWER                                                   31
A CHINESE VILLAGER WHO SELLS MORE SOFTWARE DAILY THAN YOU DO    31
DON'T ASK WHY YOU WERE REJECTED                                 31
YOUR HIGH IQ WILL KILL YOUR STARTUP                             31
My Experience In HTML                                           28
Welcome to Lucid                                                27
with picture                                                    27
This is lucid                                                   27
this is a test post                                             27
Your face PHP script                                            27
this is mine                                                    27
html test                                                       27
Lucid is live                                                   14
Testing tags and really long posts                            

In [11]:
user_df['post_id'].value_counts()

998.0     111
371.0      90
992.0      84
1007.0     75
985.0      48
993.0      48
360.0      39
986.0      24
983.0      24
355.0      16
375.0      15
996.0      12
353.0      12
999.0       6
1002.0      6
995.0       4
439.0       3
143.0       3
278.0       2
Name: post_id, dtype: int64

In [12]:
user_df['Ratings'].value_counts()

3    256
5    152
4    147
2     67
Name: Ratings, dtype: int64

In [13]:
count_by_rating = pd.DataFrame(user_df.groupby(['user_id'])['Ratings'].value_counts())
count_by_rating.sort_values('Ratings', ascending = False)
count_by_rating.head(20)

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: 'Ratings' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  


Ratings
user_id Ratings         
1       4              8
        5              8
        2              4
2       5            132
        4            108
        3             84
3       3            135
        2             60
        4             15
4       3              4
        4              4
6       3              3
7       3             24
        4             12
        5             12
22      2              3
        3              3
2191    3              3

In [14]:
count_by_title = pd.DataFrame(user_df.groupby(['user_id'])['title'].value_counts())
count_by_title.sort_values('title', ascending = False)
count_by_title.head(20)

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: 'title' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  


title
user_id title                                                    
1       First                                                   5
        Took a long while                                       5
        second                                                  5
        third                                                   5
2       A CHINESE VILLAGER WHO SELLS MORE SOFTWARE DAIL...     27
        BUILD AROUND YOUR DISCOVERY MECHANISM                  27
        DON'T ASK WHY YOU WERE REJECTED                        27
        THE SUNFLOWER                                          27
        This is lucid                                          27
        Welcome to Lucid                                       27
        YOUR HIGH IQ WILL KILL YOUR STARTUP                    27
        Your face PHP script                                   27
        html test                                              27
        this is a test post                                    27
        this is mine                                           27
        with picture                                           27
3       My Experience In HTML                                  28
        Class Day 1                                            14
        Hello New Followers                                    14
        How did your day go?                                   14

In [15]:
count_by_postid = pd.DataFrame(user_df.groupby(['user_id'])['post_id'].value_counts())
count_by_postid.sort_values('post_id', ascending = False)
count_by_postid.head(20)

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: 'post_id' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  


post_id
user_id post_id         
1       996.0         12
        355.0          4
        995.0          4
2       998.0         96
        992.0         84
        985.0         48
        993.0         48
        983.0         24
        986.0         24
3       371.0         90
        1007.0        75
        360.0         15
        375.0         15
        998.0         15
4       999.0          6
        278.0          2
6       439.0          3
7       360.0         24
        353.0         12
        355.0         12